In [25]:
import os
import yaml

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
metric_PATH = './'

In [ ]:
# reading metrics
def read_metrics(path):
    with open(path, 'r') as f:
        metrics = yaml.load(f)
    mA = metrics['mA']
    acc = metrics['acc']
    prec = metrics['prec']
    recall = metrics['recall']
    f1 = metrics['f1']
    return mA, acc, prec, recall, f1